# Feature Importance Code

In [ ]:
# Panel Predictive R-squared

import numpy as np

def reduction_in_r2_importance(model, X, y):
    """Calculates feature importance using reduction in panel predictive R².

    Args:
        model (fitted model): The trained machine learning model.
        X (pd.DataFrame or np.array): The matrix of features.
        y (pd.Series or np.array): The target variable.

    Returns:
        pd.Series: Feature importances as reductions in R².
    """
    
    original_r2 = R_oos(model.predict(X), y)

    importances = []
    for col in X.columns:
        X_copy = X.copy()
        X_copy[col] = 0  # Set all values of the feature to zero
        reduced_r2 = R_oos(model.predict(X_copy), y)
        importance = original_r2 - reduced_r2
        importances.append(importance)

    return pd.Series(importances, index=X.columns)


In [ ]:
# Sum of Squared Partial Derivatives (SSD) for Neural Networks

import torch

def ssd_importance(model, X):
    """Calculates feature importance using sum of squared partial derivatives (SSD).

    Args:
        model (PyTorch model): The trained neural network model.
        X (torch.Tensor): The matrix of features as a PyTorch tensor.

    Returns:
        torch.Tensor: Feature importances as SSD values.
    """

    X.requires_grad = True  # Enables calculation of gradients

    predictions = model(X)
    importances = []
    for i in range(predictions.shape[1]):  # Assume output is per-feature
        gradients = torch.autograd.grad(predictions[:, i], X, create_graph=True)[0]
        importance = torch.sum(gradients ** 2, dim=0)  # Sum of squared gradients over samples
        importances.append(importance.detach().numpy())  # Convert to NumPy

    return torch.tensor(importances)


In [ ]:
# Mean Decrease in Impurity (MDI) for Random Forests and Boosted Trees

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor()
rf.fit(X, y)
importances = rf.feature_importances_
